# Classifying MNIST digits using Logistic Regression

This notebook will show how Theano can be used to implement the logistic regression. As the plan, we have,

1) A brief Intro to logistic Regression

2) Loading data models

3) Making the Logistic Regression model

4) Running the model

Almost all of the material here can be found [here](http://deeplearning.net/tutorial/). This work is entirly baed on this tutorial.

## The model

So, What is Logistic regression?

Logistic regression is a probabilistic, linear classifier. It is parametrized by a weight matrix $W$ and a bias vector $b$. Classification is done by projecting an input vector onto a set of hyperplanes, each of which corresponds to a class. The distance from the input to a hyperplane reflects the probability that the input is a member of the corresponding class.
Mathematically, the probability that an input vector $x$ is a member of a class $i$, a value of a stochastic variable $Y$, can be written as:
\begin{align}
P(Y = i \mid x, W,b) &= softmax_i (Wx+b)\\
&= \frac{e^{W_i x + b_i}}{\sum_{j}{{e}^{W_j x + b_j }}}
\end{align}


The model’s prediction $y_{pred}$ is the class whose probability is maximal, specifically:
\begin{equation}
y_{pred} = argmax_i P(Y = i \mid x,W,b)
\end{equation}


 We thus maximize the log-likelihood of our classifier given all the labels in a training set.
\begin{equation}
\mathcal{L}(\theta, \mathcal{D}) =
    \sum_{i=0}^{|\mathcal{D}|} \log P(Y=y^{(i)} | x^{(i)}, \theta)
\end{equation}
The likelihood of the correct class is not the same as the number of right predictions, but from the point of view of a randomly initialized classifier they are pretty similar. 

Since we usually speak in terms of minimizing a loss function, learning will thus attempt to minimize the negative log-likelihood (NLL).

## Stochastic Gradient Descent
What is ordinary gradient descent? it is a simple algorithm in which we repeatedly make small steps downward on an error surface defined by a loss function of some parameters. For the purpose of ordinary gradient descent we consider that the training data is rolled into the loss function.

Stochastic gradient descent (SGD) works according to the same principles as ordinary gradient descent, but proceeds more quickly by estimating the gradient from just a few examples at a time instead of the entire training set. In its purest form, we estimate the gradient from just a single example at a time.

The variant that is generally recommended for deep learning is a further twist on stochastic gradient descent using so-called “minibatches”. Minibatch SGD (MSGD) works identically to SGD, except that we use more than one training example to make each estimate of the gradient. This technique reduces variance in the estimate of the gradient, and often makes better use of the hierarchical memory organization in modern computers.

## Regularization
while training our model from data, we are trying to prepare it to do well on new examples, not the ones it has already seen. The training loop above for MSGD does not take this into account, and may overfit the training examples. A way to combat overfitting is through regularization.

#### L2 regularization
L2 regularization involve adding an extra term to the loss function, which penalizes certain parameter configurations.

If our loss function is
\begin{equation}
NLL(\theta, \mathcal{D}) = - \sum_{i=0}^{|\mathcal{D}|} \log P(Y=y^{(i)} | x^{(i)}, \theta)
\end{equation}
then the regularized loss will be:
\begin{equation}
E(\theta, \mathcal{D}) =  NLL(\theta, \mathcal{D}) + \lambda||\theta||_2^2
\end{equation}
where, $||\theta||$ is the $L_2$ norm of $\theta$.

## Loading the data

You can either download the data from [Kaggle](https://www.kaggle.com/c/digit-recognizer/data), or from [The MNIST Database website](http://yann.lecun.com/exdb/mnist/).

However, to understand the use of pickled data, we will load it from the existing picked file, [mnist.pkl.gz](mnist.pkl.gz)

## Pickle 

Pickle is used for serializing and de-serializing a Python object structure. Any object in python can be pickled so that it can be saved on disk. For more information, check [this](https://pythontips.com/2013/08/02/what-is-pickle-in-python/) out.

#### Now lets get to it!

In [5]:
# Importing the useful libraries and packages

from __future__ import print_function

__docformat__ = 'restructedtext en'

import six.moves.cPickle as pickle
import gzip
import os
import sys
import timeit
import csv
import numpy

import theano
import theano.tensor as T


We need to first define the Logistic regression class.
It will have some basic definations,

1) <code>\_\_init\_\_</code> : for the initialization. According the the number of inputs and outputs.

2) <code> negative_log_likelihood </code> : For calculating the mean loss with respect to a given set of points of outcome variables(Minibatch).

3)<code> error </code> : This gives the proportion of incorrectly labelled points.

Kindly check the file, [LogisticRegression.py](code/LogisticRegression.py) to see how it has been coded. We will simply import it.

In [26]:
from code import LogisticRegression as LR

Now, we will start the main processing.

The steps involved are

1) first

2) second

In [27]:
# First we set some hyper parameters.
learning_rate = 0.13
n_epochs = 1000
datasets = "mnist.pkl.gz"
batch_size = 600

In [55]:
import csv
csv_file = csv.reader(open("train.csv"))
header = csv_file.next()
data = []
for row in csv_file:
    data.append(row)
data_full = numpy.array(data, dtype = "float32")
datax = data_full[:,1:]
datay = data_full[:,0:1]
train_x = datax[0:37000,:]
train_y = datay[0:37000,:]
train_y.shape =(int(numpy.shape(train_y)[0]))
validate_x =datax[37000:,:]
validate_y = datay[37000:,:]
validate_y.shape =(5000)
train_set_x = theano.shared(numpy.asarray(train_x,
                        dtype=theano.config.floatX),
                        borrow=True)
train_set_y = theano.shared(numpy.asarray(train_y,
                        dtype=theano.config.floatX),
                        borrow=True)
train_set_y = T.cast(train_set_y,'int32')
valid_set_x = theano.shared(numpy.asarray(validate_x,
                        dtype=theano.config.floatX),
                        borrow=True)
valid_set_y = theano.shared(numpy.asarray(validate_y,
                        dtype=theano.config.floatX),
                        borrow=True)
valid_set_y = T.cast(valid_set_y,'int32')

In [56]:
n_train_batches = train_set_x.get_value(borrow=True).shape[0] // batch_size
n_valid_batches = valid_set_x.get_value(borrow=True).shape[0] // batch_size

In [57]:
print(n_train_batches)

61


In [58]:
print(n_valid_batches)

8


### Model Building

In [59]:
index = T.lscalar()
x = T.matrix('x')
y = T.ivector('y')
classifier = LR.LogisticRegression(input=x, n_in = 28*28 , n_out = 10)
# You can check different features of the class.
#classifier.input

In [60]:
# We set the total Loss/ the cost to be the negative Log-likelihood of the function
cost = classifier.negative_log_likelihood(y)

In [61]:
type(valid_set_y)

theano.tensor.var.TensorVariable

In [62]:
# Now we make a function to find the error we make in the mini batches we take
#test_model = theano.function(
#        inputs=[index],
#        outputs=classifier.errors(y),
#        givens={
#            x: test_set_x[index * batch_size: (index + 1) * batch_size],
#            y: test_set_y[index * batch_size: (index + 1) * batch_size]
#        }
#    )
# similarly
validate_model = theano.function(
        inputs=[index],
        outputs=classifier.errors(y),
        givens={
            x: valid_set_x[index * batch_size: (index + 1) * batch_size],
            y: valid_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )

In [63]:
# compute the gradient of cost with respect to theta = (W,b)
g_W = T.grad(cost=cost, wrt=classifier.W)
g_b = T.grad(cost=cost, wrt=classifier.b)

# For looking at the derivative

In [64]:
# Now, for the update step
# where classifier.W and classifier.b are shared theano tensors.

updates = [(classifier.W, classifier.W - learning_rate * g_W),(classifier.b, classifier.b - learning_rate * g_b)]

In [65]:
# This is the coolest part.
# compiling a Theano function `train_model` that returns the cost, but in
# the same time updates the parameter of the model based on the rules
# defined in `updates`
train_model = theano.function(
    inputs=[index],
    outputs=cost,
    updates=updates,
    givens={
        x: train_set_x[index * batch_size: (index + 1) * batch_size],
        y: train_set_y[index * batch_size: (index + 1) * batch_size]
    }
)
# We have defined all the necessary functions.

In [66]:
# Now we define the stopping criteria based on patientce.
# early-stopping parameters
patience = 5000  # look as this many examples regardless
patience_increase = 2  # wait this much longer when a new best is
                       # found
improvement_threshold = 0.995  # a relative improvement of this much is
                              # considered significant
validation_frequency = min(n_train_batches, patience // 2)
                              # go through this many
                              # minibatche before checking the network
                              # on the validation set; in this case we
                              # check every epoch

In [67]:
# Initializing the parameters
best_validation_loss = numpy.inf
test_score = 0.
start_time = timeit.default_timer()
done_looping = False
epoch = 0

In [69]:
while (epoch < n_epochs) and (not done_looping):
        epoch = epoch + 1
        for minibatch_index in range(n_train_batches):

            minibatch_avg_cost = train_model(minibatch_index)
            # iteration number
            iter = (epoch - 1) * n_train_batches + minibatch_index

            if (iter + 1) % validation_frequency == 0:
                # compute zero-one loss on validation set
                validation_losses = [validate_model(i)
                                     for i in range(n_valid_batches)]
                this_validation_loss = numpy.mean(validation_losses)

                print(
                    'epoch %i, minibatch %i/%i, validation error %f %%' %
                    (
                        epoch,
                        minibatch_index + 1,
                        n_train_batches,
                        this_validation_loss * 100.
                    )
                )

                # if we got the best validation score until now
                if this_validation_loss < best_validation_loss:
                    #improve patience if loss improvement is good enough
                    if this_validation_loss < best_validation_loss *  \
                       improvement_threshold:
                        patience = max(patience, iter * patience_increase)

                    best_validation_loss = this_validation_loss
                    # test it on the test set

                    #test_losses = [test_model(i)
                    #               for i in range(n_test_batches)]
                    #test_score = numpy.mean(test_losses)
                    #print(
                    #    (
                    #        '     epoch %i, minibatch %i/%i, test error of'
                    #        ' best model %f %%'
                    #    ) %
                    #    (
                    #        epoch,
                    #        minibatch_index + 1,
                    #        n_train_batches,
                    #        test_score * 100.
                    #    )
                    #)
                    # save the best model
                    with open('best_model.pkl', 'wb') as f:
                        pickle.dump(classifier, f)
                    print("Better then previous best function!")
                    
            if patience <= iter:
                done_looping = True
                break

end_time = timeit.default_timer()
print(
    (
        'Optimization complete with best validation score of %f %%.'
    )
    % (best_validation_loss * 100.)
)
print('The code run for %d epochs, with %f epochs/sec' % (
    epoch, 1. * epoch / (end_time - start_time)))
print(('The code for file ' +
       os.path.split('__file__')[1] +
       ' ran for %.1fs' % ((end_time - start_time))), file=sys.stderr)

epoch 63, minibatch 61/61, validation error 9.750000 %
epoch 64, minibatch 61/61, validation error 10.208333 %
epoch 65, minibatch 61/61, validation error 9.208333 %
epoch 66, minibatch 61/61, validation error 9.791667 %
epoch 67, minibatch 61/61, validation error 9.916667 %
epoch 68, minibatch 61/61, validation error 12.541667 %
epoch 69, minibatch 61/61, validation error 16.187500 %
epoch 70, minibatch 61/61, validation error 9.187500 %
epoch 71, minibatch 61/61, validation error 9.166667 %
epoch 72, minibatch 61/61, validation error 9.354167 %
epoch 73, minibatch 61/61, validation error 9.312500 %
epoch 74, minibatch 61/61, validation error 9.520833 %
epoch 75, minibatch 61/61, validation error 9.687500 %
epoch 76, minibatch 61/61, validation error 9.104167 %
epoch 77, minibatch 61/61, validation error 9.791667 %
epoch 78, minibatch 61/61, validation error 9.604167 %
epoch 79, minibatch 61/61, validation error 10.354167 %
epoch 80, minibatch 61/61, validation error 21.083333 %
epoch

The code for file __file__ ran for 214.2s


### Inference
So, we made a Logistic regression model, and updated its parameter based on min-batch Stochastic gradient descent.
After 74 epochs, we could get the best test performance of roughly 93%. The model training reached the stopping criteria in about 2 minutes.

We can now use this model for predicting the values on unseen datasets.Below is a function which can be used for prediction.

In [70]:
def predict(shared_xx):
    """
    An example of how to load a trained model and use it
    to predict labels.
    """

    # load the saved model
    #classifier = pickle.load(open('best_model.pkl'))

    # compile a predictor function
    predict_model = theano.function(
        inputs=[classifier.input],
        outputs=classifier.y_pred)

    # We can test it on some examples from test test
    #dataset='mnist.pkl.gz'
    #datasets = dl.load_data(dataset)
    #test_set_x, test_set_y = datasets[2]
    #test_set_x = test_set_x.get_value()

    #predicted_values = predict_model(test_set_x[:100])
    #xx = theano.shared(data)
    predicted_values = predict_model(shared_xx.get_value())
    return list(predicted_values)

## Trying the Model on Kaggle competition

Now, we will try to see the model accuracy on the dataset in kaggle. It contains 28000 images in the test set. As we used mode in training, a lot of the data would be exact points in our training data. This would lead to a buffed up result, but it would also act as a relative metric while comparing this model to the others we will be making, specifically multi-layer preceptron and Convoluted neural network.

In [74]:
#loading the data
csv_file = csv.reader(open("test.csv"))
header = csv_file.next()
data = []
for row in csv_file:
    data.append(row)
datax = numpy.array(data, dtype = "float32")
test_x = theano.shared(numpy.asarray(data,
                        dtype=theano.config.floatX),
                        borrow=True)

In [75]:
# Using the predict() function
test_labels = predict(test_x)

# We need a serial ID array as well
image_id = numpy.arange(1,len(test_labels)+1)

#Now to write it to a csv file
f = open("submission2.csv", "w")
f.write("{},{}\n".format("ImageId", "Label"))
for x in zip(image_id, test_labels):
    f.write("{},{}\n".format(x[0], x[1]))
f.close()

## Result
We find the result to be 86% accurate. This is great for a model that gets trained in just two minutes.